In [ ]:
# Knapsack problem solved with a Genetic Algorithm
w = []
v = []
W = 1

import ipywidgets as widgets


def create_button():
    button = widgets.Button(description='Next Generation',
                            disabled=False, button_style='',
                            tooltip='Next Generation', icon='check')  # 'success', 'info', 'warning', 'danger' or ''
                                                                      # (FontAwesome names without the `fa-` prefix)
    return button

def create_button_move10():
  button = widgets.Button(description = "Move 10 generations",
                          disabled = False, button_style = '',
                          tooltip = 'Move 10 generations',
                          icon = 'check')
  return button


import math
import random
import time
import matplotlib.pyplot as plt
import numpy as np
from IPython import display as display

from functools import cmp_to_key

#Fill randomly v and w
def createRandomItems():
  for i in range(0, 20):
      w.append(random.random())
      v.append(random.randrange(1, 101))

createRandomItems()

#createRandomProblem()

# Chromosomes are 4 bits long
L_chromosome = len(v)
N_chains = 2 ** L_chromosome

# Number of chromosomes
N_chromosomes = 50

# probability of mutation
prob_m = 0.5

crossover_point = int(L_chromosome / 2)

def random_chromosome():
    chromosome = []
    for i in range(0, L_chromosome):
        if random.random() < 0.1:
            chromosome.append(0)
        else:
            chromosome.append(1)

    return chromosome


F0 = []
fitness_values = []

for i in range(0, N_chromosomes):
    F0.append(random_chromosome())
    fitness_values.append(0)

def decode_chromosome(chromosome):
    global L_chromosome, v, w

    Total_weight = sum([w_i * c_i for (w_i, c_i) in zip(w, chromosome)])
    Total_value = sum([v_i * c_i for (v_i, c_i) in zip(v, chromosome)])

    return (Total_value, Total_weight)

def f(x):
    global W
    (Total_value, Total_weight) = x
    excess = Total_weight - W
    return (Total_value if excess <= 0 else Total_value - excess * 1000)


def evaluate_chromosomes():
    global F0

    for p in range(N_chromosomes):
        v = decode_chromosome(F0[p])
        fitness_values[p] = f(v)


def compare_chromosomes(chromosome1, chromosome2):
    vc1 = decode_chromosome(chromosome1)
    vc2 = decode_chromosome(chromosome2)
    fvc1 = f(vc1)
    fvc2 = f(vc2)

    if fvc1 < fvc2:
        return 1
    elif fvc1 == fvc2:
        return 0
    else: # fvg1<fvg2
        return -1


suma = float(N_chromosomes * (N_chromosomes + 1)) / 2.

Lwheel = N_chromosomes * 10


def create_wheel():
    global F0, fitness_values

    maxv = max(fitness_values)
    acc = 0
    for p in range(N_chromosomes):
        acc += maxv - fitness_values[p]
    if acc == 0:
        return [0] * Lwheel
    fraction = []
    for p in range(N_chromosomes):
        fraction.append(float(maxv - fitness_values[p]) / acc)
        if fraction[-1] <= 1.0 / Lwheel:
            fraction[-1] = 1.0 / Lwheel

##    print fraction
    fraction[0] -= (sum(fraction) - 1.0) / 2
    fraction[1] -= (sum(fraction) - 1.0) / 2

##    print fraction
    wheel = []

    pc = 0

    for f in fraction:
        Np = int(f * Lwheel)
        for i in range(Np):
            wheel.append(pc)
        pc += 1

    return wheel


F1 = F0[:]
n = 0
def nextgeneration(b):
    global n
    display.clear_output(wait=True)
    display.display(button)
    display.display(button_move10_generations)

    F0.sort(key=cmp_to_key(compare_chromosomes))
    #print(F0)
    #print(fitness_values)
    print('Best solution so far:')
    n += 1
    print (
        n,
        F0[0],
        'f(',
        decode_chromosome(F0[0]),
        ')= ',
        f(decode_chromosome(F0[0])),
        )

    # elitism, the two best chromosomes go directly to the next generation

    F1[0] = F0[0]
    F1[1] = F0[1]
    roulette = create_wheel()

    # print (roulette)

    for i in range(0, int((N_chromosomes - 2) / 2)):

        # Two parents are selected

        p1 = random.choice(roulette)
        p2 = random.choice(roulette)

        # Two descendants are generated

        o1 = (F0[p1])[0:crossover_point]
        o1.extend((F0[p2])[crossover_point:L_chromosome])
        o2 = (F0[p2])[0:crossover_point]
        o2.extend((F0[p1])[crossover_point:L_chromosome])

        # Each descendant is mutated with probability prob_m

        if random.random() < prob_m:
            o1[int(round(random.random() * (L_chromosome - 1)))] ^= 1
        if random.random() < prob_m:
            o2[int(round(random.random() * (L_chromosome - 1)))] ^= 1

        # The descendants are added to F1

        F1[2 + 2 * i] = o1
        F1[3 + 2 * i] = o2

    # The generation replaces the old one

    F0[:] = F1[:]
    evaluate_chromosomes()

def move10_generations(param):
  for i in range(0, 10):
    nextgeneration(param)

xmax = 400
ymax = 400

button = create_button()
button.on_click(nextgeneration)
button_move10_generations = create_button_move10()
button_move10_generations.on_click(move10_generations)

display.display(button)
display.display(button_move10_generations)

F0.sort(key=cmp_to_key(compare_chromosomes))
evaluate_chromosomes()


Button(description='Next Generation', icon='check', style=ButtonStyle(), tooltip='Next Generation')

Button(description='Move 10 generations', icon='check', style=ButtonStyle(), tooltip='Move 10 generations')

Best solution so far:
300 [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0] f( (304, 0.9553031578463539) )=  304


In [ ]:
#Traveling salesman problem with a GA

import math
import random
import numpy as np
from IPython import display as display
from functools import cmp_to_key
import ipywidgets as widgets

#Number of chromosomes
nChromosomes = 100

#Probablity of mutation
probM = 0.5

#Chromosomes list
F0 = []

#Chromosomes fitness value
fitnessValues = []

#Remember we are solving the traveling salesman problem so we need to use distances between cities
#Create the adjacency matrix
cities = [[0,8782,3779,5390,5550,1589,8835], 
          [8782,0,5549,6853,5982,7195,693],
          [3779,5549,0,7820,7228,2574,5299],
          [5390,6853,7820,0,985,5621,7546],
          [5550,5982,7228,985,0,5382,6672],
          [1589,7195,2574,5621,5382,0,7261],
          [8835,693,5299,7546,6672,7261,0]]

def createButton():
  button = widgets.Button(
      description = "Next generation",
      disabled = False,
      button_style = '',
      tooltip = 'Next generation',
      icon = 'check'
  )
  return button

def createButton10Generations():
  button = widgets.Button(
      description = "Move 10 generations",
      disabled = False,
      button_style = '',
      tooltip = 'Move 10 generations',
      icon = 'check'
  )
  return button

def nextPermutation(arr):
  arrCopy = arr.copy()
  wasFound = False
  #At least we need two numbers to get a permutation
  i = len(arrCopy) - 2
  while i >= 0:
    if arrCopy[i] < arrCopy[i + 1]:
      wasFound = True
      break
    i -= 1

  if not wasFound:
    arrCopy.sort()
  else:
    pivotIndex = findPivot(arrCopy, arrCopy[i])
    arrCopy[i], arrCopy[pivotIndex] = arrCopy[pivotIndex], arrCopy[i] 
    arrCopy[i + 1:] = arrCopy[i + 1:][:: -1]
  return arrCopy

def findPivot(arr, element):
  ans = -1
  index = 0
  for i in range(index, len(arr)):
    if arr[i] > element:
      if ans == -1:
        ans = element
        index = i
      else:
        ans = min(ans, arr[i])
        index = i
  return index
  
listOfPossibleChromosomes = [[0, 1, 2, 3, 4, 5, 6]]

def generateAllPermutations(numberOfNodes):
  #Each city is a node of a graph the total number of permutations is given by n! where n is the number of nodes
  numberOfPermutations = math.factorial(numberOfNodes)
  #We get all the permutations
  for i in range(0, numberOfPermutations):
    chromosome = nextPermutation(listOfPossibleChromosomes[i])
    listOfPossibleChromosomes.append(chromosome)

def getAllChromosomes(numberOfNodes):
  upperBound = math.factorial(numberOfNodes)
  for i in range(0, nChromosomes):
    F0.append(listOfPossibleChromosomes[random.randrange(0, upperBound)])
    fitnessValues.append(0)

generateAllPermutations(7)
getAllChromosomes(7)

#We get the total distance
def f(x):
  distancePath = 0
  for i in range (0, len(x) - 1):
    distance = cities[x[i]][x[i + 1]]
    if distance != 0:
      distancePath += distance
    else:
      distancePath = math.inf
      break
  return distancePath

#Or in other words our fitness function
def evaluateChromosomes():
  global F0
  for p in range(nChromosomes):
    fitnessValues[p] = f(F0[p])

def compareChromosomes(chromosome1, chromosome2):
  fResultChromosome1 = f(chromosome1)
  fResultChromosome2 = f(chromosome2)

  if fResultChromosome1 > fResultChromosome2:
    return 1
  elif fResultChromosome1 == fResultChromosome2:
    return 0
  #fResultChromosome1 < fResultChromosome2 otherwise
  return -1

#Function that makes the crossover process using the PMX algorithm
def crossover(ancestor1, ancestor2):
  #We are "spliting" our chromosomes in two parts of random size in order to "mix" each part of the information
  crossPoint1 = random.randint(0, len(ancestor1) - 2)
  crossPoint2 = random.randint(crossPoint1 + 1, len(ancestor1) - 1)
  #Now we make a copy of the "left piece" of the chromosomes received
  middleCrossA1 = ancestor1[crossPoint1:crossPoint2]
  middleCrossA2 = ancestor2[crossPoint1:crossPoint2]
  middleCrossSize = crossPoint2 - crossPoint1
  #Creating partial new chromosomes creating a relationship between the pieces that we got from crossPoint1 and crossPoint2
  #And mixing them with the other ancestor graphicaly something like (x x x | a b c | x x)
  partialChild1 = ancestor1[:crossPoint1] + middleCrossA2 + ancestor1[crossPoint2:]
  partialChild2 = ancestor2[:crossPoint1] + middleCrossA1 + ancestor2[crossPoint2:]
  #We can use a data structure to know as fast as possible if the element that we choose is already in use (A set)
  valuesAlreadyExist1 = set()
  valuesAlreadyExist2 = set()
  relations = {}
  for index in range(0, middleCrossSize):
    u = middleCrossA1[index]
    v = middleCrossA2[index]
    valuesAlreadyExist1.add(v)
    valuesAlreadyExist2.add(u)
    #We add the relations between elements to find as fast as possible wich element we can use
    if not u in relations:
      relations[u] = [v]
    else:
      relations[u].append(v)

    if not v in relations:
      relations[v] = [u]
    else:
      relations[v].append(u)
  
  def getValidChild(partialChild, valuesAlreadyExist):
    partialChildSize = len(partialChild)
    #Checking if the current chromosome contains unique and valid values
    for index in range(0, partialChildSize):
      if (index < crossPoint1 or index >= crossPoint2):
        currElement = partialChild[index]

        #For this function we'll use a similar idea to a BFS to try to find the valid element that we can use
        #But we are asuming that for ever we will find a valid solution or in other words
        #We're asuming that the relations have an "end"
        def findValidValue(key):
          if key in valuesAlreadyExist:
            queue = []
            queue.append(key)

            while len(queue) != 0:
              u = queue.pop(0)
              for v in relations[u]:
                if not v in valuesAlreadyExist:
                  valuesAlreadyExist.add(v)
                  return v
                queue.append(v)
          return key
        
        validValue = findValidValue(currElement)
        partialChild[index] = validValue
    return partialChild
    
  validChild1 = getValidChild(partialChild1, valuesAlreadyExist1)
  validChild2 = getValidChild(partialChild2, valuesAlreadyExist2)

  return validChild1, validChild2

#It is a sum using the gauss sum
suma = float(nChromosomes * (nChromosomes + 1)) / 2
lWheel = nChromosomes * 10

def createWheel():
  global F0, fitnessValues
  maxValue = max(fitnessValues)
  accumulator = 0
  for p in range(nChromosomes):
    accumulator += maxValue - fitnessValues[p]
  if accumulator == 0:
    return [0]*lWheel
  fraction = []
  for p in range(nChromosomes):
    fraction.append(float(maxValue - fitnessValues[p]) / accumulator)
    if fraction[-1] <= 1.0 / lWheel:
      fraction[-1] = 1.0 / lWheel
  fraction[0] -= (math.fsum(fraction)-1.0) / 2
  fraction[1] -= (math.fsum(fraction)-1.0) / 2

  wheel = []
  pc = 0
  for f in fraction:
    Np = int(f * lWheel)
    for i in range(Np):
      wheel.append(pc)
    pc += 1
  return wheel

F1 = F0[:]
n = 0

#Generate the next generation of chromosomes
def nextGeneration(b):
    global n
    display.clear_output(wait = True)
    display.display(button)
    display.display(button10Generations)
    F0.sort(key = cmp_to_key(compareChromosomes) )
    print( "Best solution so far:")
    n += 1
    print( n,F0[0],"\nf(",F0[0],") = ", f(F0[0]) )
                                                                    
    F1[0]=F0[0]
    F1[1]=F0[1]

    roulette = createWheel()

    for i in range(0, int((nChromosomes- 2 ) / 2)):      
    
        ancestor1 = random.choice(roulette)
        ancestor2 = random.choice(roulette)
    
        offsprings = crossover(F0[ancestor1], F0[ancestor2])
        offspring1 = offsprings[0]
        offspring2 = offsprings[1]

        if random.random() < probM:
            pos = random.randrange(0,7)
            pos2 =  random.randrange(0,7)
            offspring1[pos], offspring1[pos2] = offspring1[pos2], offspring1[pos]
        if random.random() < probM:
            pos = random.randrange(0,7)
            pos2 =  random.randrange(0,7)
            offspring2[pos], offspring2[pos2] = offspring2[pos2], offspring2[pos]
        F1[2 + 2 * i] = offspring1
        F1[3 + 2 * i] = offspring2

    F0[:] = F1[:]
    evaluateChromosomes()


def move10Generations(param):
  for i in range(0, 10):
    nextGeneration(param)

button = createButton()
button.on_click(nextGeneration)
display.display(button)

button10Generations = createButton10Generations()
button10Generations.on_click(move10Generations)
display.display(button10Generations)


F0.sort( key=cmp_to_key(compareChromosomes) )
evaluateChromosomes()  

Button(description='Next generation', icon='check', style=ButtonStyle(), tooltip='Next generation')

Button(description='Move 10 generations', icon='check', style=ButtonStyle(), tooltip='Move 10 generations')

Best solution so far:
100 [4, 3, 0, 5, 2, 6, 1] 
f( [4, 3, 0, 5, 2, 6, 1] ) =  16530
